# Imports

In [ ]:
import os
import cv2
import random
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Util Code

In [ ]:
def count_images_per_class(data_path):
    class_counts = {}
    for tumor_type in os.listdir(data_path):
        class_dir = os.path.join(data_path, tumor_type)
        if os.path.isdir(class_dir):
            num_images = len(glob.glob(f"{class_dir}/*.jpg"))
            class_counts[tumor_type] = num_images
            print(f"{tumor_type}: {num_images} images")
    return class_counts

def plot_random_images_per_class(data_path, num_samples=1, figsize=(12, 8)):
    classes = os.listdir(data_path)
    plt.figure(figsize=figsize)

    for i, tumor_type in enumerate(classes):
        tumor_dir = os.path.join(data_path, tumor_type)
        image_files = os.listdir(tumor_dir)
        if not image_files:
            continue
        random_image = random.choice(image_files)
        image_path = os.path.join(tumor_dir, random_image)
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        plt.subplot(2, 2, i+1)
        plt.imshow(img)
        plt.title(tumor_type)
        plt.axis('off')
    plt.tight_layout()
    plt.show()


def create_generators(train_dir,
                      test_dir,
                      img_size=(224, 224),
                      batch_size=32,
                      preprocessing=None,
                      val_split=0.2,
                      shuffle_test=False):
    """
    Creates train, validation, and test data generators.

    Args:
        train_dir (str): Path to training directory.
        test_dir (str): Path to testing directory.
        img_size (tuple): Desired image size (height, width).
        batch_size (int): Number of images per batch.
        preprocessing (function): Optional preprocessing function (e.g., preprocess_input).
        val_split (float): Fraction of training data to use for validation.
        shuffle_test (bool): If False, test generator won't shuffle files (important for eval consistency).

    Returns:
        train_gen, val_gen, test_gen: Keras ImageDataGenerator iterators
    """
    if preprocessing:
        train_val_datagen = ImageDataGenerator(
            preprocessing_function=preprocessing,
            validation_split=val_split
        )
        test_datagen = ImageDataGenerator(preprocessing_function=preprocessing)
    else:
        train_val_datagen = ImageDataGenerator(rescale=1./255, validation_split=val_split)
        test_datagen = ImageDataGenerator(rescale=1./255)

    train_gen = train_val_datagen.flow_from_directory(
        train_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='sparse',
        subset='training'
    )

    val_gen = train_val_datagen.flow_from_directory(
        train_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='sparse',
        subset='validation'
    )

    test_gen = test_datagen.flow_from_directory(
        test_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='sparse',
        shuffle=shuffle_test
    )

    return train_gen, val_gen, test_gen

In [ ]:
import os
from tensorflow.keras.models import load_model

def save_model_to_drive(model, model_name='model', drive_path='/content/drive/MyDrive/BrainTumorClassification/saved_models'):

    """
    Saves a Keras model to Google Drive in `.keras` format.

    Args:
        model: Trained Keras model.
        model_name (str): Name to save the file as.
        drive_path (str): Destination folder path in Google Drive.
    """
    os.makedirs(drive_path, exist_ok=True)
    filepath = os.path.join(drive_path, f"{model_name}.keras")
    model.save(filepath)
    print(f"✅ Model saved to Drive at: {filepath}")


def load_model_from_drive(model_name='model', drive_path='/content/drive/MyDrive/saved_models'):
    """
    Loads a `.keras` model from Google Drive.

    Args:
        model_name (str): Name of the file to load (without extension).
        drive_path (str): Folder path in Google Drive.

    Returns:
        Loaded Keras model
    """
    filepath = os.path.join(drive_path, f"{model_name}.keras")
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"No model found at {filepath}")
    print(f"📦 Loading model from: {filepath}")
    return load_model(filepath)


In [ ]:
def get_augmented_train_generator(train_dir, img_size=(224, 224), batch_size=32):
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=15,
        width_shift_range=0.1,
        height_shift_range=0.1,
        brightness_range=(0.8, 1.2),
        zoom_range=0.2,
        shear_range=0.1,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    train_gen = train_datagen.flow_from_directory(
        train_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='sparse',
        shuffle=True
    )

    return train_gen
